# Rigid Body Simulator

Credit to Professor Bo Zhu, on whose COSC 89 Course Notes the following code is based on

## Geometry in space

UNFINISHED

Question: by adding a third dimension to all the vectors to allow for cross products, the direction of gravity must be flipped to align with our frame of reference (ie now gravity is (0, 2, 0). why is this?

In [ ]:
import taichi as ti
import numpy as np
import math

ti.init()

# variables and fields to keep track of
num_particles = 1
dt = 1e-3
substeps = 10
mass = 1

I = 10 # Inertial tensor
x = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles) # position of center of mass
vertices_0 = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles*3) # vertices of triangles relative to COM
rotated_vertices = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles*3)
force = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)
torque = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles)
v = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles) # velocity
theta = ti.field(dtype=ti.f32, shape=num_particles) # orientation
w = ti.Vector.field(n=3, dtype=ti.f32, shape=num_particles) # angular velocity

@ti.kernel
def init_pos():    
    x[0] = ti.Vector([0.5, 0.5, 0])
    # equilateral triangle
    vertices_0[0] = ti.Vector([0, 0.1, 0])
    vertices_0[1] = ti.Vector([-0.0866, -0.05, 0])
    vertices_0[2] = ti.Vector([0.0866, -0.05, 0])
    rotated_vertices[0] = ti.Vector([0, 0.1, 0])
    rotated_vertices[1] = ti.Vector([-0.0866, -0.05, 0])
    rotated_vertices[2] = ti.Vector([0.0866, -0.05, 0])
    ### this is assuming a one object system
    theta[0] = math.pi/12
    v[0] = ti.Vector([0, 0, 0]) # init velocity
    force[0] = ti.Vector([0,0,0]) # init force
    torque[0] = ti.Vector([0,0,0]) 
    w[0] = ti.Vector([0, 0, 1]) # small amount of angular to start
    
@ti.func
def clear_force():
    for i in range(num_particles):
        force[i] = ti.Vector([0.0, 0.0, 0.0])
    
@ti.func
def calculate_force():
    clear_force()
    # gravity
    for i in range(num_particles):
        force[i] = force[i] - ti.Vector([0, 0, 0])

@ti.func
def rotate():
    t = theta[0]
    R = ti.Vector([[ti.cos(t), -ti.sin(t), 0], [ti.sin(t), ti.cos(t), 0], [0, 0, 0]])
    for i in range(3):
        vec = ti.Vector([vertices_0[i][0], vertices_0[i][1], vertices_0[i][2]])
        result = R @ vec
        rotated_vertices[i] = ti.Vector([result[0], result[1], result[2]])

@ti.func
def calculate_torque():
    # no torque from gravity since r = 0
    pass

@ti.kernel
def substep():
    calculate_force()
    calculate_torque()
    # update velocity
    v[0] = v[0] + dt * force[0] / mass
    # update angular velocity
    w[0] = w[0] + dt * torque[0] / I
    # update position
    x[0] = x[0] + dt * v[0]
    # update orientation
    theta[0] = theta[0] + dt * w[0][2]
    # update vertices
    rotate()

def main():
    gui = ti.GUI('Rigid Body', (600, 600))
    # initialize vertices
    init_pos()
        
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            pass
            
        # Update for a time step
        for step in range(substeps):
            substep()

        print(theta[0])
        gui.triangle(a=x[0]+rotated_vertices[0], b=x[0]+rotated_vertices[1], c=x[0]+rotated_vertices[2], color=0xED553B)
        
        gui.show()
    
if __name__ == '__main__':
    main()

[Taichi] Starting on arch=x64
0.2717992663383484
0.2817991375923157
0.29179900884628296
0.30179888010025024
0.31179875135421753
0.3217986226081848
0.3317984938621521
0.3417983651161194
0.35179823637008667
0.36179810762405396
0.37179797887802124
0.3817978501319885
0.3917977213859558
0.4017975926399231
0.4117974638938904
0.42179733514785767
0.43179720640182495
0.44179707765579224
0.4517969489097595
0.4617968201637268
0.4717966914176941
0.4817965626716614
0.49179643392562866
0.501796305179596
0.5117961764335632
0.5217960476875305
0.5317959189414978
0.5417957901954651
0.5517956614494324
0.5617955327033997
0.5717954039573669
0.5817952752113342
0.5917951464653015
0.6017950177192688
0.6117948889732361
0.6217947602272034
0.6317946314811707
0.6417945027351379
0.6517943739891052
0.6617942452430725
0.6717941164970398
0.6817939877510071
0.6917938590049744
0.7017937302589417
0.7117936015129089
0.7217934727668762
0.7317933440208435
0.7417932152748108
0.7517930865287781
0.7617929577827454
0.771792829

4.691641807556152
4.701641082763672
4.711640357971191
4.721639633178711
4.7316389083862305
4.74163818359375
4.7516374588012695
4.761636734008789
4.771636009216309
4.781635284423828
4.791634559631348
4.801633834838867
4.811633110046387
4.821632385253906
4.831631660461426
4.841630935668945
4.851630210876465
4.861629486083984
4.871628761291504
4.881628036499023
4.891627311706543
4.9016265869140625
4.911625862121582
4.921625137329102
4.931624412536621
4.941623687744141
4.95162296295166
4.96162223815918
4.971621513366699
4.981620788574219
4.991620063781738
5.001619338989258
5.011618614196777
5.021617889404297
5.031617164611816
5.041616439819336
5.0516157150268555
5.061614990234375
5.0716142654418945
5.081613540649414
5.091612815856934
5.101612091064453
5.111611366271973
5.121610641479492
5.131609916687012
5.141609191894531
5.151608467102051
5.16160774230957
5.17160701751709
5.181606292724609
5.191605567932129
5.201604843139648
5.211604118347168
5.2216033935546875
5.231602668762207
5.2416019

Code from stackoverflow on finding the inertial tensor using triangles

https://stackoverflow.com/questions/41592034/computing-tensor-of-inertia-in-2d